In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
tvdeskDF = pd.read_csv('./project/tv_desk.csv', index_col=0)
tvdeskDF['image'] 

0        ./static/img/living/tv_desk/tv_desk1.png
1        ./static/img/living/tv_desk/tv_desk2.png
2        ./static/img/living/tv_desk/tv_desk3.png
3        ./static/img/living/tv_desk/tv_desk4.png
4        ./static/img/living/tv_desk/tv_desk5.png
                          ...                    
100    ./static/img/living/tv_desk/tv_desk101.png
101    ./static/img/living/tv_desk/tv_desk102.png
102    ./static/img/living/tv_desk/tv_desk103.png
103    ./static/img/living/tv_desk/tv_desk104.png
104    ./static/img/living/tv_desk/tv_desk105.png
Name: image, Length: 105, dtype: object

In [5]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(tvdeskDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

tvdeskDF['nouns'] = box

100%|██████████| 105/105 [00:20<00:00,  5.07it/s]


In [6]:
tvdeskDF.head(1)

,brand,item,price,image,link,review,nouns
0,지벤,노르마니 LED 익스텐션 모던 거실장,"159,000",./static/img/living/tv_desk/tv_desk1.png,https://ohou.se/productions/346800/selling?aff...,['딱 원하던 컬러의 거실장이었는데 디자인도 마음에 들고 튼튼하고 라이트까지 들어와...,컬러 거실 디자인 마음 라이트 고민 바로 구매 배치 생각 아주 마음 길이 조절 수 ...


In [8]:
# 중간 저장
tvdeskDF.to_csv('tvdeskDF.csv')

In [10]:
tvdeskDF = pd.read_csv('./tvdeskDF.csv', index_col=0)
tvdeskDF.head(1)

,brand,item,price,image,link,review,nouns
0,지벤,노르마니 LED 익스텐션 모던 거실장,"159,000",./static/img/living/tv_desk/tv_desk1.png,https://ohou.se/productions/346800/selling?aff...,['딱 원하던 컬러의 거실장이었는데 디자인도 마음에 들고 튼튼하고 라이트까지 들어와...,컬러 거실 디자인 마음 라이트 고민 바로 구매 배치 생각 아주 마음 길이 조절 수 ...


In [13]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(tvdeskDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_tvdesk = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_tvdesk

,가걱,가걱만킁,가게,가격,가공,가구,가구라,가구점,가기,가까이,...,희롱,희망,흰색,힌지,힐링,힘글었어,힘껏,힘들엇어,힙니,힙함
0,0,0,0,16,0,7,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,4,0,3,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,1,24,0,3,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,25,0,5,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,0,0,0,3,0,13,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
101,0,0,0,9,0,9,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
102,0,0,0,9,0,5,0,0,0,2,...,0,0,1,0,0,0,0,0,0,0
103,0,0,0,5,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [14]:
# dtm 파일 저장
dtm_tvdesk.to_csv('dtm_tvdesk.csv')

---

In [16]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

tvdeskDF = pd.read_csv('./tvdeskDF.csv', index_col=0)
dtm_tvdesk = pd.read_csv('./dtm_tvdesk.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = 'tv가 절대 떨어지지 않을 견고한 스타일'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(tvdeskDF['brand'])
item_list = np.array(tvdeskDF['item'])
price_list = np.array(tvdeskDF['price'])
image_list = np.array(tvdeskDF['image'])

result_cosine = make_cosine(dtm_tvdesk, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

베이직가구 블랙러버 AE형 거실장 399,000 ./static/img/living/tv_desk/tv_desk41.png


In [ ]:
# 위의 코드 함수화
def find_tvdesk(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    tvdeskDF = pd.read_csv('./tvdeskDF.csv', index_col=0)
    dtm_tvdesk = pd.read_csv('./dtm_tvdesk.csv', index_col=0)   

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(tvdeskDF['brand'])
    item_list = np.array(tvdeskDF['item'])
    price_list = np.array(tvdeskDF['price'])
    image_list = np.array(tvdeskDF['image'])

    result_cosine = make_cosine(dtm_tvdesk, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]